In [1]:
# Import required librariesimport pandas as pdimport numpy as npimport pickleimport refrom sklearn.preprocessing import LabelEncoderfrom sklearn.model_selection import train_test_splitfrom tensorflow.keras.models import Sequentialfrom tensorflow.keras.layers import Embedding, LSTM, Dense, Dropoutfrom tensorflow.keras.models import load_modelprint("Libraries imported successfully!")

In [2]:
# Load the Swiggy datasetdata = pd.read_csv('smartapp/swiggy.csv')print(f"Loaded {len(data)} records from swiggy.csv")print(f"Columns: {data.columns.tolist()}")

In [3]:
# Preprocess datadata["Review"] = data["Review"].str.lower()data["Review"] = data["Review"].replace(r'[^a-z0-9\s]', '', regex=True)data = data.dropna(subset=['Review', 'Avg Rating'])def label_sentiment_num(rating):    if rating <= 2.5:        return "negative"    elif rating <= 3.5:        return "neutral"    else:        return "positive"data['sentiment'] = data['Avg Rating'].apply(label_sentiment_num)print(f"Original sentiment distribution:")print(data['sentiment'].value_counts())

In [4]:
# Check if we have negative examplesnegative_count = len(data[data['sentiment'] == 'negative'])if negative_count == 0:    print("No negative examples found! Adding synthetic negative examples...")        # Common negative phrases for restaurant reviews    negative_phrases = [        "terrible food and bad service",        "worst experience ever",        "cold and tasteless food",        "disgusting and overpriced",        "horrible delivery and rude staff",        "never order from here again",        "waste of money and time",        "food was terrible and cold",        "very bad service and slow",        "disappointed with the quality",        "tasteless and bland food",        "late delivery and wrong order",        "poor quality and expensive",        "awful experience and rude",        "bad taste and dirty place",        "not fresh and overcooked",        "unhygienic and unhealthy",        "worse than expected and cold",        "cheap quality and small portions",        "unpleasant and frustrating service"    ]        # Create synthetic negative examples    synthetic_negative = pd.DataFrame({        'Review': negative_phrases * 100,        'sentiment': ['negative'] * (len(negative_phrases) * 100),        'Avg Rating': [2.0] * (len(negative_phrases) * 100)    })        # Add to original data    data = pd.concat([data[['Review', 'sentiment', 'Avg Rating']], synthetic_negative], ignore_index=True)    print("After adding synthetic negatives:")    print(data['sentiment'].value_counts())else:    print(f"Found {negative_count} negative examples")

In [5]:
# Build vocabularymax_features = 5000max_length = 200def build_vocab(texts, max_features=None):    word_counts = {}    for text in texts:        for word in str(text).split():            word_counts[word] = word_counts.get(word, 0) + 1        sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)        if max_features:        sorted_words = sorted_words[:max_features]        word_index = {word: idx+1 for idx, (word, _) in enumerate(sorted_words)}    return word_indexword_index = build_vocab(data["Review"].tolist(), max_features=max_features)print(f"Vocabulary size: {len(word_index)}")

In [6]:
# Convert texts to sequencesdef texts_to_sequences(texts, word_index):    sequences = []    for text in texts:        seq = [word_index.get(word, 0) for word in str(text).split()]        sequences.append(seq)    return sequencessequences = texts_to_sequences(data["Review"].tolist(), word_index)# Pad sequencesdef pad_sequences_custom(sequences, maxlen):    padded = []    for seq in sequences:        if len(seq) < maxlen:            seq = [0]*(maxlen - len(seq)) + seq        else:            seq = seq[:maxlen]        padded.append(seq)    return np.array(padded)X = pad_sequences_custom(sequences, max_length)y = data['sentiment'].valuesprint(f"X shape: {X.shape}, y shape: {y.shape}")

In [7]:
# Split dataX_train, X_test, y_train, y_test = train_test_split(    X, y, test_size=0.2, random_state=42, stratify=y)X_train, X_val, y_train, y_val = train_test_split(    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)# Encode labelsencoder = LabelEncoder()y_train_encoded = encoder.fit_transform(y_train)y_val_encoded = encoder.transform(y_val)y_test_encoded = encoder.transform(y_test)print(f"Label classes: {encoder.classes_}")print(f"Train distribution: {np.bincount(y_train_encoded)}")print(f"Test distribution: {np.bincount(y_test_encoded)}")

In [8]:
# Build LSTM modelembedding_dim = 128model = Sequential([    Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=max_length),    LSTM(128, return_sequences=True),    Dropout(0.3),    LSTM(64),    Dropout(0.3),    Dense(64, activation="relu"),    Dense(3, activation="softmax")])model.compile(    loss="sparse_categorical_crossentropy",    optimizer="adam",    metrics=["accuracy"])model.summary()

In [9]:
# Train the modelprint("Training model...")history = model.fit(    X_train, y_train_encoded,    validation_data=(X_val, y_val_encoded),    epochs=10,    batch_size=64)# Evaluateloss, acc = model.evaluate(X_test, y_test_encoded)print(f"Test Accuracy: {acc:.2f}")

In [10]:
# Save the model and artifactsprint("Saving model and artifacts...")# Save Keras modelmodel.save('smartapp/sentiment_model.h5')print("Saved: smartapp/sentiment_model.h5")# Save word_indexwith open('smartapp/word_index.pkl', 'wb') as f:    pickle.dump(word_index, f)print("Saved: smartapp/word_index.pkl")# Save label encoderwith open('smartapp/label_encoder.pkl', 'wb') as f:    pickle.dump(encoder, f)print("Saved: smartapp/label_encoder.pkl")# Save max_length for later usewith open('smartapp/model_config.pkl', 'wb') as f:    pickle.dump({'max_length': max_length, 'max_features': max_features}, f)print("Saved: smartapp/model_config.pkl")

In [11]:
# Test the saved modelprint("Testing saved model...")# Load saved artifactsloaded_model = load_model('smartapp/sentiment_model.h5')with open('smartapp/word_index.pkl', 'rb') as f:    loaded_word_index = pickle.load(f)with open('smartapp/label_encoder.pkl', 'rb') as f:    loaded_encoder = pickle.load(f)# Test reviewstest_reviews = [    "The food was amazing and delicious!",    "Terrible service and cold food",    "It was okay, nothing special",    "Best restaurant ever, highly recommend!",    "Worst experience ever, never coming back",    "Fresh and tasty, will order again",    "Hate the long wait and rude staff",    "Average quality but decent"]print("Test Results:")for review in test_reviews:    # Preprocess    text = review.lower().strip()    text = re.sub(r'[^\w\s]', '', text)    words = text.split()    sequence = [loaded_word_index.get(word, 0) for word in words]    if len(sequence) < max_length:        sequence = [0] * (max_length - len(sequence)) + sequence    else:        sequence = sequence[:max_length]    sequence = np.array([sequence])        # Predict    pred = loaded_model.predict(sequence, verbose=0)[0]    pred_class = np.argmax(pred)    sentiment = loaded_encoder.inverse_transform([pred_class])[0]    confidence = float(pred[pred_class] * 100)        print(f"Review: '{review}'")    print(f"  Sentiment: {sentiment} ({confidence:.1f}%)")print("Model training and export complete!")